## Get the TensorRT tar file before running this Notebook

1. Visit https://developer.nvidia.com/tensorrt
2. Clicking `Download now` from step one directs you to https://developer.nvidia.com/nvidia-tensorrt-download where you have to Login/Join Now for Nvidia Developer Program Membership
3. Now, in the download page: Choose TensorRT 8 in available versions
4. Agree to Terms and Conditions
5. Click on TensorRT 8.5 GA to expand the available options
6. Click on 'TensorRT 8.5 GA for Linux x86_64 and CUDA 11.0, 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7 and 11.8 TAR Package' to dowload the TAR file
7. Upload the the tar file to your Google Drive

## Connect to GPU Instance

1. Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accelerator)
1. Then click on Connect (Top Right)

## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [ ]:
import sys
if 'google.colab' in sys.modules:
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
else:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

# License Plate Recognition using TAO LPRNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained baseline18 LPRNet model and train it on the OpenALPR benchmark dataset
* Run Inference on the trained model
* Export the trained model to a .etlt file for deployment to DeepStream

## Table of Contents

This notebook shows an example usecase of LPRNet using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables](#head-0)
1. [Prepare dataset and pre-trained model](#head-1) <br>
    1.1 [Download pre-trained model](#head-1-1) <br>
2. [Setup GPU environment](#head-2) <br>
    2.1 [Setup Python environment](#head-2-1) <br>
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Inferences](#head-6)

#### Note
1. This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly
2. This notebook uses OPENALPR dataset by default, which should be around ~2.2 MB.
3. Using the default config/spec file provided in this notebook, each weight file size of lprnet created during training will be ~111 MB

## 0. Set up env variables and set FIXME parameters <a class="anchor" id="head-0"></a>

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

#### FIXME
1. NUM_GPUS - set this to <= number of GPU's availble on the instance
1. GPU_INDEX - set to to the indices of the GPU available on the instance
1. COLAB_NOTEBOOKS_PATH - for Google Colab environment, set this path where you want to clone the repo to; for local system environment, set this path to the already cloned repo
1. EXPERIMENT_DIR - set this path to a folder location where pretrained models, checkpoints and log files during different model actions will be saved
1. delete_existing_experiments - set to True to remove existing pretrained models, checkpoints and log files of a previous experiment
1. DATA_DIR - set this path to a folder location where you want to dataset to be present
1. delete_existing_data - set this to True to remove existing preprocessed and original data
1. trt_tar_path - set this path of the uploaded TensorRT tar.gz file after browser download
1. trt_untar_folder_path - set to path of the folder where the TensoRT tar.gz file has to be untarred into
1. trt_version - set this to the version of TRT you have downloaded

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env TAO_DOCKER_DISABLE=1

%env KEY=nvidia_tlt
#FIXME1
%env NUM_GPUS=1
#FIXME2
%env GPU_INDEX=0

#FIXME3
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/nvidia-tao
if os.environ["GOOGLE_COLAB"] == "1":
    if not os.path.exists(os.path.join(os.environ["COLAB_NOTEBOOKS_PATH"])):

      !git clone https://github.com/NVIDIA-AI-IOT/nvidia-tao.git $COLAB_NOTEBOOKS_PATH
else:
    if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
        raise Exception("Error, enter the path of the colab notebooks repo correctly")

#FIXME4
%env EXPERIMENT_DIR=/content/drive/MyDrive/results/lprnet
#FIXME5
delete_existing_experiments = True
#FIXME6
%env DATA_DIR=/content/drive/MyDrive/lprnet_data/
#FIXME7
delete_existing_data = False

if delete_existing_experiments:
    !sudo rm -rf $EXPERIMENT_DIR
if delete_existing_data:
    !sudo rm -rf $DATA_DIR

SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/lprnet/specs"
%env SPECS_DIR={SPECS_DIR}
# Showing list of specification files.
!ls -rlt $SPECS_DIR

!sudo mkdir -p $DATA_DIR && sudo chmod -R 777 $DATA_DIR
!sudo mkdir -p $EXPERIMENT_DIR && sudo chmod -R 777 $EXPERIMENT_DIR

## 1. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

 We will be using the OpenALPR benchmark dataset for the tutorial. The following script will download the dataset automatically and convert it to the format used by TAO. 

In [ ]:
!bash $COLAB_NOTEBOOKS_PATH/tensorflow/lprnet/download_and_prepare_data.sh $DATA_DIR

In [ ]:
# verify
!echo $DATA_DIR
!ls -l $DATA_DIR/
!ls -l $DATA_DIR/train
!ls -l $DATA_DIR/train/image
!ls -l $DATA_DIR/train/label


### 1.1 Download pretrained model from NGC

We will use NGC CLI to get the pre-trained models. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env LOCAL_PROJECT_DIR=/ngc_content/
%env CLI=ngccli_cat_linux.zip
!sudo mkdir -p $LOCAL_PROJECT_DIR/ngccli && sudo chmod -R 777 $LOCAL_PROJECT_DIR

# Remove any previously existing CLI installations
!sudo rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u -q "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 $LOCAL_PROJECT_DIR/ngccli/ngc-cli/libstdc++.so.6

In [ ]:
!ngc registry model list nvidia/tao/lprnet:*

In [ ]:
!mkdir -p $EXPERIMENT_DIR/pretrained_lprnet_baseline18/

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/lprnet:trainable_v1.0 --dest $EXPERIMENT_DIR/pretrained_lprnet_baseline18

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $EXPERIMENT_DIR/pretrained_lprnet_baseline18/lprnet_vtrainable_v1.0

## 2. Setup GPU environment <a class="anchor" id="head-2"></a>


### 2.1 Setup Python environment <a class="anchor" id="head-2-1"></a>
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
# FIXME 8: set this path of the uploaded TensorRT tar.gz file after browser download
trt_tar_path="/content/drive/MyDrive/TensorRT-8.5.1.7.Linux.x86_64-gnu.cuda-11.8.cudnn8.6.tar.gz"

import os
if not os.path.exists(trt_tar_path):
  raise Exception("TAR file not found in the provided path")

# FIXME 9: set to path of the folder where the TensoRT tar.gz file has to be untarred into
%env trt_untar_folder_path=/content/trt_untar
# FIXME 10: set this to the version of TRT you have downloaded
%env trt_version=8.5.1.7

!mkdir -p $trt_untar_folder_path

import os

untar = True
for fname in os.listdir(os.environ.get("trt_untar_folder_path", None)):
  if fname.startswith("TensorRT-"+os.environ.get("trt_version")) and not fname.endswith(".tar.gz"):
    untar = False

if untar:
  !tar -xzf $trt_tar_path -C /content/trt_untar

In [ ]:
import os
if os.environ["GOOGLE_COLAB"] == "1":
    os.environ["bash_script"] = "setup_env.sh"
else:
    os.environ["bash_script"] = "setup_env_desktop.sh"

!sed -i "s|PATH_TO_TRT|$trt_untar_folder_path|g"$COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script
!sed -i "s|TRT_VERSION|$trt_version|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script
!sed -i "s|PATH_TO_COLAB_NOTEBOOKS|$COLAB_NOTEBOOKS_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script

!sh $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script

## 3. Provide training specification <a class="anchor" id="head-3"></a>

* Note the spec $SPEC_DIR/default_sepc.txt is for training on US license plates:
    * the max license plate length is 8;
        * You can change `max_label_length` in `lpr_config` to satisfy your own dataset.
    * the characters of US license plates are: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, C, D, E, F, G, H, I, J, K, L, M, N, P, Q, R, S, T, U, V, W, X, Y, Z 
        * You can change `characters_list_file` in `dataset_config` to set your own characters.
        * `characters_list_file` should contain all the characters in dataset. And one character takes one line. 

In [ ]:
!sed -i "s|TAO_DATA_PATH|$DATA_DIR/|g" $SPECS_DIR/tutorial_spec.txt
!sed -i "s|TAO_SPEC_DIR|$SPECS_DIR/|g" $SPECS_DIR/tutorial_spec.txt
!cat $SPECS_DIR/tutorial_spec.txt

In [ ]:
!cat $SPECS_DIR/us_lp_characters.txt

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [ ]:
!mkdir -p $EXPERIMENT_DIR/experiment_dir_unpruned

In [ ]:
print("For multi-GPU, change --gpus based on your machine.")
!tao model lprnet train --gpus=1 --gpu_index=$GPU_INDEX \
                  -e $SPECS_DIR/tutorial_spec.txt \
                  -r $EXPERIMENT_DIR/experiment_dir_unpruned \
                  -k $KEY \
                  -m $EXPERIMENT_DIR/pretrained_lprnet_baseline18/lprnet_vtrainable_v1.0/us_lprnet_baseline18_trainable.tlt

In [ ]:
print("To resume training from a checkpoint, set the -m option to be the .tlt you want to resume from and --initial_epochs to be the epoch index of the resumed checkpoint")
# !tao model lprnet train --gpu_index=$GPU_INDEX \
#                   -e $SPECS_DIR/tutorial_spec.txt \
#                   -r $EXPERIMENT_DIR/experiment_dir_unpruned \
#                   -k $KEY \
#                   -m $EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-01.tlt
#                   --initial_epoch 2

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $EXPERIMENT_DIR/experiment_dir_unpruned/weights/

## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

In [ ]:
!tao model lprnet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/tutorial_spec.txt \
                     -m $EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-24.tlt \
                     -k $KEY

## 6. Inferences <a class="anchor" id="head-6"></a>
In this section, we run the lprnet inference tool to generate inferences on the trained models and print the results. 

In [ ]:
# Running inference for detection on n images
!tao model lprnet inference --gpu_index=$GPU_INDEX -i $DATA_DIR/val/image \
                      -e $SPECS_DIR/tutorial_spec.txt \
                      -m $EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-24.tlt \
                      -k $KEY 